# 🏡 Min-Max Normalization Workshop
## Team Name: 2
## Team Members: Manu Mathew, Parth, Kumari Nikitha
---

## ❗ Why We Normalize: The Problem with Raw Feature Scales

In housing data, features like `Price` and `Lot_Size` can have values in the hundreds of thousands, while others like `Num_Bedrooms` range from 1 to 5. This creates problems when we use algorithms that depend on numeric magnitudes.

---

### ⚠️ What Goes Wrong Without Normalization

---

### 1. 🧭 K-Nearest Neighbors (KNN)

KNN uses the **Euclidean distance** formula:

$$
d = \sqrt{(x_1 - x_2)^2 + (y_1 - y_2)^2 + \cdots}
$$

**Example:**

- $ \text{Price}_1 = 650{,}000, \quad \text{Price}_2 = 250{,}000 $
- $ \text{Bedrooms}_1 = 3, \quad \text{Bedrooms}_2 = 2 $

Now compute squared differences:

$$
(\text{Price}_1 - \text{Price}_2)^2 = (650{,}000 - 250{,}000)^2 = (400{,}000)^2 = 1.6 \times 10^{11}
$$
$$
(\text{Bedrooms}_1 - \text{Bedrooms}_2)^2 = (3 - 2)^2 = 1
$$

➡️ **Price dominates the distance calculation**, making smaller features like `Bedrooms` irrelevant.

---

### 2. 📉 Linear Regression

Linear regression estimates:

$$
y = \beta_1 \cdot \text{Price} + \beta_2 \cdot \text{Bedrooms} + \beta_3 \cdot \text{Lot\_Size} + \epsilon
$$

If `Price` has very large values:
- Gradient updates for $ \beta_1 $ will be **much larger**
- Gradient updates for $ \beta_2 $ (Bedrooms) will be **very small**

➡️ The model overfits high-magnitude features like `Price`.

---

### 3. 🧠 Neural Networks

A single neuron computes:

$$
z = w_1 \cdot \text{Price} + w_2 \cdot \text{Bedrooms} + w_3 \cdot \text{Lot\_Size}
$$

If:

- $ \text{Price} = 650{,}000 $
- $ \text{Bedrooms} = 3 $
- $ \text{Lot\_Size} = 8{,}000 $

Then:

$$
z \approx w_1 \cdot 650{,}000 + w_2 \cdot 3 + w_3 \cdot 8{,}000
$$

➡️ Even with equal weights, `Price` contributes **most of the activation**, making it difficult for the network to learn from other features.

---

### ✅ Solution: Min-Max Normalization

We apply the transformation:

$$
x_{\text{normalized}} = \frac{x - x_{\text{min}}}{x_{\text{max}} - x_{\text{min}}}
$$

This scales all features to a common range (typically $[0, 1]$).

| Feature      | Raw Value | Min     | Max     | Normalized Value |
|--------------|-----------|---------|---------|------------------|
| Price        | 650,000   | 250,000 | 800,000 | 0.72             |
| Bedrooms     | 3         | 1       | 5       | 0.50             |
| Lot_Size     | 8,000     | 3,000   | 10,000  | 0.714            |

➡️ Now, **each feature contributes fairly** to model training or distance comparisons.

---

## 📌 Use Case: Housing Data
We are normalizing features from a real estate dataset to prepare it for machine learning analysis.

In [13]:
# 🔢 Load and display dataset
import pandas as pd
df = pd.read_csv('./data/housing_data.csv')
df.head()

,House_ID,Price,Area_sqft,Num_Bedrooms,Num_Bathrooms,Year_Built,Lot_Size
0,H100000,574507,1462,3,3,2002,4878
1,H100001,479260,1727,2,2,1979,4943
2,H100002,597153,1403,5,2,1952,5595
3,H100003,728454,1646,5,2,1992,9305
4,H100004,464876,853,1,1,1956,7407


# Data Cleaning  and Exploratory Data Analysis

In [14]:
# checking for missing values
print(df.isnull().sum())

# checking for duplicates
print(df.duplicated().sum())

House_ID         0
Price            0
Area_sqft        0
Num_Bedrooms     0
Num_Bathrooms    0
Year_Built       0
Lot_Size         0
dtype: int64
0


There is no missing or null values in the data set. Also there is 

In [15]:
# displaying basic information about the DataFrame
print("DataFrame Information:")
df.info()

# displaying descriptive statistics
print("\n Descriptive Statistics:")
df.describe()

DataFrame Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   House_ID       2000 non-null   object
 1   Price          2000 non-null   int64 
 2   Area_sqft      2000 non-null   int64 
 3   Num_Bedrooms   2000 non-null   int64 
 4   Num_Bathrooms  2000 non-null   int64 
 5   Year_Built     2000 non-null   int64 
 6   Lot_Size       2000 non-null   int64 
dtypes: int64(6), object(1)
memory usage: 109.5+ KB

 Descriptive Statistics:


,Price,Area_sqft,Num_Bedrooms,Num_Bathrooms,Year_Built,Lot_Size
count,2.000000e+03,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,5.068961e+05,1796.453000,2.983500,1.966000,1985.689500,6025.246000
std,1.478786e+05,502.185109,1.409333,0.825945,21.159536,2008.527265
min,1.000000e+05,400.000000,1.000000,1.000000,1950.000000,1000.000000
25%,4.066002e+05,1445.000000,2.000000,1.000000,1967.000000,4664.000000
50%,5.067030e+05,1799.500000,3.000000,2.000000,1986.000000,6010.500000
75%,6.024458e+05,2132.000000,4.000000,3.000000,2003.000000,7414.000000
max,1.077909e+06,3763.000000,5.000000,3.000000,2022.000000,13088.000000


### 🔎 Step 1 — Implement Min-Max Normalization on the Housing Dataset

In [ ]:
# ✍️ Implement Min-Max Normalization manually here (no sklearn/numpy)
# Normalize: Price, Area_sqft, Num_Bedrooms, Num_Bathrooms, Lot_Size

# Columns to normalize
cols = ["Price", "Area_sqft", "Num_Bedrooms", "Num_Bathrooms", "Lot_Size"]

# Pre-compute minima and maxima
mins = {c: df[c].min() for c in cols}
maxs = {c: df[c].max() for c in cols}
print("Minima:", mins)
print("Maxima:", maxs)

In the above steps what we are doing is creating a list of the column names we are going to normalize. After this we will create a 
 a dictionary for both the min and max that looks like {'Price': 1000000, 'Area_sqft': 501 ...}. These dictionaries will hold the min and max
 for the corresponding column name.

In [ ]:


# Helper that returns a normalized Series
def minmax(col):
    min, max = mins[col], maxs[col]
    if min == max:  
        print(f"Min is equal to Max for {col}, normalization skipped.")         
        return
    return (df[col] - min) / (max - min)



Minima: {'Price': np.int64(100000), 'Area_sqft': np.int64(400), 'Num_Bedrooms': np.int64(1), 'Num_Bathrooms': np.int64(1), 'Lot_Size': np.int64(1000)}
Maxima: {'Price': np.int64(1077909), 'Area_sqft': np.int64(3763), 'Num_Bedrooms': np.int64(5), 'Num_Bathrooms': np.int64(3), 'Lot_Size': np.int64(13088)}


,House_ID,Price,Area_sqft,Num_Bedrooms,Num_Bathrooms,Year_Built,Lot_Size,Price_norm,Area_sqft_norm,Num_Bedrooms_norm,Num_Bathrooms_norm,Lot_Size_norm
0,H100000,574507,1462,3,3,2002,4878,0.485226,0.315789,0.50,1.0,0.320814
1,H100001,479260,1727,2,2,1979,4943,0.387827,0.394588,0.25,0.5,0.326191
2,H100002,597153,1403,5,2,1952,5595,0.508384,0.298246,1.00,0.5,0.380129
3,H100003,728454,1646,5,2,1992,9305,0.642651,0.370503,1.00,0.5,0.687045
4,H100004,464876,853,1,1,1956,7407,0.373119,0.134701,0.00,0.0,0.530030
5,H100005,464879,1906,1,3,1998,7617,0.373122,0.447814,0.00,1.0,0.547402
6,H100006,736881,1800,3,1,2022,9215,0.651268,0.416295,0.50,0.0,0.679600
7,H100007,615115,1391,2,2,1990,7805,0.526751,0.294677,0.25,0.5,0.562955
8,H100008,429578,2129,4,1,2003,3737,0.337023,0.514124,0.75,0.0,0.226423
9,H100009,581384,2268,1,3,2018,7774,0.492258,0.555456,0.00,1.0,0.560390


Here we created a helper function minmax(col): -> float, this helper function simply applies the normalization 
equation based on the provided column name and the corresponding min and max values. It also ensures that our min and max values
are not equal, as the nature of the equation used will result in division by 0 if the min and max values are equal.

In [ ]:
# Add normalized columns (suffix “_norm”); drop or rename as you like
for c in cols:
    df[c + "_norm"] = minmax(c)
df.head(10)

# df now contains both original and normalized values

In this last for loop we will add a new column <columnname>_norm for each of the normalized columns

df now contains both original and normalized values

### 🔎 Talking Point 1 — Why normalization is needed

Reviwed by:
- Haysam
- Tracy
- Benny